## Load the required csv 

In [3]:
import pandas as pd

inventories_fy23_path = "inventories_full_fy23.csv"
inventories_fy24_path = "inventories_full_fy24.csv"

inventories_fy23 = pd.read_csv(inventories_fy23_path)
inventories_fy24 = pd.read_csv(inventories_fy24_path)

In [4]:
# inventories_fy23.head()

In [5]:
# inventories_fy24.head()

## Connecting to our local neo4j graph

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Create the Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Test the connection
try:
    with driver.session() as session:
        result = session.run("RETURN 1")
        if result.single()[0] == 1:
            print("Neo4j connection successful!")
        else:
            print("Neo4j connection failed!")
except Exception as e:
    print(f"Neo4j connection error: {e}")
finally:
    driver.close()


Neo4j connection successful!


In [7]:
# from neo4j import GraphDatabase

# # URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
# URI = "neo4j+s://669a1604.databases.neo4j.io"
# AUTH = ("neo4j", "aVhj0fbW8GaSkgk8aFjh3i2QmhIUWKnfnt2yCvJu-yk")

# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     driver.verify_connectivity()
# driver.close()

## Adding Nodes and Relationships to our Neo4j KG

In [8]:
def add_hierarchy_relationships_outline(driver, df):
    current_L1 = current_L2 = current_L3 = None

    with driver.session() as session:
        for idx, row in df.iterrows():
            l1 = str(row['L1']).strip() if pd.notna(row['L1']) and row['L1'] else None
            l2 = str(row['L2']).strip() if pd.notna(row['L2']) and row['L2'] else None
            l3 = str(row['L3']).strip() if pd.notna(row['L3']) and row['L3'] else None
            l4 = str(row['L4']).strip() if pd.notna(row['L4']) and row['L4'] else None

            if l1:
                current_L1 = l1
                current_L2 = current_L3 = None  # Reset lower levels
                print(f"Found L1: {l1}")
            elif l2:
                current_L2 = l2
                current_L3 = None
                # L1 must exist
                session.run("""
                    MERGE (a:InventoryLevel1 {name: $l1})
                    MERGE (b:InventoryLevel2 {name: $l2})
                    MERGE (a)-[:HAS_SUB_INVENTORY]->(b)
                """, l1=current_L1, l2=current_L2)
                print(f"Linked L1: {current_L1} -> L2: {current_L2}")
            elif l3:
                current_L3 = l3
                # L2 must exist
                session.run("""
                    MERGE (a:InventoryLevel2 {name: $l2})
                    MERGE (b:InventoryLevel3 {name: $l3})
                    MERGE (a)-[:HAS_SUB_INVENTORY]->(b)
                """, l2=current_L2, l3=current_L3)
                print(f"Linked L2: {current_L2} -> L3: {current_L3}")
            elif l4:
                # Prefer linking to L3, else L2, else L1
                if current_L3:
                    session.run("""
                        MERGE (a:InventoryLevel3 {name: $l3})
                        MERGE (b:InventoryLevel4 {name: $l4})
                        MERGE (a)-[:HAS_SUB_INVENTORY]->(b)
                    """, l3=current_L3, l4=l4)
                    print(f"Linked L3: {current_L3} -> L4: {l4}")
                elif current_L2:
                    session.run("""
                        MERGE (a:InventoryLevel2 {name: $l2})
                        MERGE (b:InventoryLevel4 {name: $l4})
                        MERGE (a)-[:HAS_SUB_INVENTORY]->(b)
                    """, l2=current_L2, l4=l4)
                    print(f"Linked L2: {current_L2} -> L4: {l4}")
                elif current_L1:
                    session.run("""
                        MERGE (a:InventoryLevel1 {name: $l1})
                        MERGE (b:InventoryLevel4 {name: $l4})
                        MERGE (a)-[:HAS_SUB_INVENTORY]->(b)
                    """, l1=current_L1, l4=l4)
                    print(f"Linked L1: {current_L1} -> L4: {l4}")




In [9]:
# Example usage:
add_hierarchy_relationships_outline(driver, inventories_fy23)
add_hierarchy_relationships_outline(driver, inventories_fy24)

Found L1: Inventories


C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\1591674299.py:4: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Linked L1: Inventories -> L2: Raw Material & Mfg. Componets
Linked L2: Raw Material & Mfg. Componets -> L3: Raw Material & Mfg. Componet
Linked L3: Raw Material & Mfg. Componet -> L4: Closing Stock- Gunny Bags
Linked L3: Raw Material & Mfg. Componet -> L4: Clg Stock - Raw Material-B/s
Linked L3: Raw Material & Mfg. Componet -> L4: Clg Stock - Mfg Component B/s
Linked L2: Raw Material & Mfg. Componets -> L3: Stock In Transit - Raw Material
Linked L3: Stock In Transit - Raw Material -> L4: Clg Stock In Transit - Rm - B/s
Linked L1: Inventories -> L2: Work-in-Progress
Linked L2: Work-in-Progress -> L4: WIP- Seed Potato Facility A/c
Linked L2: Work-in-Progress -> L4: Construction Work In Progress
Linked L2: Work-in-Progress -> L4: Clg Stock - Wip - B/s
Linked L2: Work-in-Progress -> L4: Clg Stock - Construction W I P - B/s
Linked L1: Inventories -> L2: Finished Goods
Linked L2: Finished Goods -> L3: Stock In Transit - Finised Goods
Linked L3: Stock In Transit - Finised Goods -> L4: Clg Sto

In [10]:
def link_groupcompany_to_inventory_outline(df, year, driver):
    group_cols = [col for col in df.columns if col not in ['L1', 'L2', 'L3', 'L4']]
    with driver.session() as session:
        for idx, row in df.iterrows():
            l4 = str(row['L4']).strip() if pd.notna(row['L4']) and row['L4'] else None
            if not l4:
                continue
            for company in group_cols:
                val = row[company]
                amount_str = str(val).strip()
                
                if amount_str in ["-", "--", "–", ""]:
                    amount = 0.0
                    # print('REPLACED -> Original: {}, replaced by: {}'.format(val, amount))
                else:
                    try:
                        amount = float(amount_str)
                        # print('TRY -> Original: {}, replaced by: {}'.format(val, amount))
                    except Exception:
                        amount = 0.0
                        # print('EXCEPT -> Original: {}, replaced by: {}'.format(val, amount))

                if amount == 0.0:
                    continue  # Only link if amount is nonzero!

                session.run("""
                    MERGE (c:GroupCompany {name: $company})
                    MERGE (inv:InventoryLevel4 {name: $l4})
                    MERGE (c)-[r:HAS_AMOUNT {year: $year}]->(inv)
                    SET r.amount = $amount
                """, company=company, l4=l4, year=year, amount=amount)
                print(f"Linked {company} to {l4} for {year}: {amount}")




In [1]:
# Example usage:
# link_groupcompany_to_inventory_outline(inventories_fy23, "FY23", driver)
# link_groupcompany_to_inventory_outline(inventories_fy24, "FY24", driver)

In [12]:
def add_inventorylevel2_and_level3_nodes_outline(driver, df):
    unique_l2 = set()
    unique_l3 = set()
    # Gather unique, non-empty L2/L3 values (ignore blanks/None)
    for val in df['L2']:
        if pd.notna(val) and str(val).strip():
            unique_l2.add(str(val).strip())
    for val in df['L3']:
        if pd.notna(val) and str(val).strip():
            unique_l3.add(str(val).strip())

    with driver.session() as session:
        for l2 in unique_l2:
            session.run("MERGE (:InventoryLevel2 {name: $name})", name=l2)
            print(f"Added InventoryLevel2: {l2}")
        for l3 in unique_l3:
            session.run("MERGE (:InventoryLevel3 {name: $name})", name=l3)
            print(f"Added InventoryLevel3: {l3}")




In [13]:
# Example usage:
add_inventorylevel2_and_level3_nodes_outline(driver, inventories_fy23)
add_inventorylevel2_and_level3_nodes_outline(driver, inventories_fy24)

Added InventoryLevel2: Stores and Spares
Added InventoryLevel2: Stock in Trade
Added InventoryLevel2: Inventory Machinery Spares
Added InventoryLevel2: Finished Goods
Added InventoryLevel2: Other Inventory
Added InventoryLevel2: Tools
Added InventoryLevel2: Work-in-Progress
Added InventoryLevel2: Raw Material & Mfg. Componets
Added InventoryLevel3: Stock In Transit - Raw Material
Added InventoryLevel3: Stock in Trade
Added InventoryLevel3: Finished Goods
Added InventoryLevel3: Stock In Transit - Finised Goods
Added InventoryLevel3: Raw Material & Mfg. Componet
Added InventoryLevel3: Stock In Transit - Traded Goods.............
Added InventoryLevel2: Stores and Spares
Added InventoryLevel2: Stock in Trade
Added InventoryLevel2: Inventory Machinery Spares
Added InventoryLevel2: Finished Goods
Added InventoryLevel2: Other Inventory
Added InventoryLevel2: Tools
Added InventoryLevel2: Work-in-Progress
Added InventoryLevel2: Raw Material & Mfg. Componets
Added InventoryLevel3: Stock In Trans

C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\3201807440.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [14]:
def test_inventory_hierarchy_paths(driver):
    query = """
    MATCH p=(:InventoryLevel1)-[:HAS_SUB_INVENTORY*]->(:InventoryLevel4)
    RETURN p LIMIT 10
    """
    with driver.session() as session:
        results = session.run(query)
        for record in results:
            path = record["p"]
            node_names = []
            for node in path.nodes:
                labels = list(node.labels)
                name = node.get("name", "[no name]")
                node_names.append(f"{labels[0]}: {name}")
            print(" -> ".join(node_names))

# Usage:
test_inventory_hierarchy_paths(driver)


C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\118950405.py:6: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


InventoryLevel1: Inventories -> InventoryLevel2: Raw Material & Mfg. Componets -> InventoryLevel3: Raw Material & Mfg. Componet -> InventoryLevel4: Closing Stock- Gunny Bags
InventoryLevel1: Inventories -> InventoryLevel2: Raw Material & Mfg. Componets -> InventoryLevel3: Raw Material & Mfg. Componet -> InventoryLevel4: Clg Stock - Raw Material-B/s
InventoryLevel1: Inventories -> InventoryLevel2: Raw Material & Mfg. Componets -> InventoryLevel3: Raw Material & Mfg. Componet -> InventoryLevel4: Clg Stock - Mfg Component B/s
InventoryLevel1: Inventories -> InventoryLevel2: Raw Material & Mfg. Componets -> InventoryLevel3: Stock In Transit - Raw Material -> InventoryLevel4: Clg Stock In Transit - Rm - B/s
InventoryLevel1: Inventories -> InventoryLevel2: Work-in-Progress -> InventoryLevel4: WIP- Seed Potato Facility A/c
InventoryLevel1: Inventories -> InventoryLevel2: Work-in-Progress -> InventoryLevel4: Construction Work In Progress
InventoryLevel1: Inventories -> InventoryLevel2: Work-in

In [15]:
def print_relationship_types(driver):
    with driver.session() as session:
        result = session.run("CALL db.relationshipTypes()")
        print("Relationship Types in DB:")
        for record in result:
            print(f" - {record[0]}")
print_relationship_types(driver)


Relationship Types in DB:
 - HAS_SUB_INVENTORY
 - HAS_AMOUNT


C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\2877436635.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [16]:
## Linking L2 and L4

def add_direct_l2_to_l4_relationships(driver):
    # List of (L2, L4) tuples as described
    direct_links = [
        # Stores and Spares
        ("Stores and Spares", "Inventory Oils, Grease & Chemicals"),
        ("Stores and Spares", "Inventorie Consumable Stores & Misc."),
        ("Stores and Spares", "G-Stock Materiali Ricoperti CBN"),
        ("Stores and Spares", "Stock in transit - Stores & Spares"),
        ("Stores and Spares", "Inventory Stores- Manual Adj"),
        ("Stores and Spares", "Inventory Packing Material"),
        # Inventory Machinery Spares
        ("Inventory Machinery Spares", "Inventory Spares"),
        ("Inventory Machinery Spares", "Inventory Manufactured Tools"),
        ("Inventory Machinery Spares", "Inventory Spares- Manual Adj"),
        ("Inventory Machinery Spares", "In Transit - Stores & Spares"),
        # Tools
        ("Tools", "Inventory Tools"),
        ("Tools", "Inventory Manufactured Tools"),
        ("Tools", "Inventory Tools- Manual Adj"),
        ("Tools", "Stock in transit - Tools"),
        ("Tools", "Inventory Dies"),
        # Other Inventory
        ("Other Inventory", "Food & Beverages"),
        ("Other Inventory", "Ind-AS  Inventories"),
        ("Other Inventory", "Plant & Machinery Held For Sale"),
    ]
    with driver.session() as session:
        for l2, l4 in direct_links:
            session.run("""
                MERGE (parent:InventoryLevel2 {name: $l2})
                MERGE (child:InventoryLevel4 {name: $l4})
                MERGE (parent)-[:HAS_SUB_INVENTORY]->(child)
            """, l2=l2, l4=l4)
            print(f"Linked L2 '{l2}' -> L4 '{l4}'")

# Usage:
add_direct_l2_to_l4_relationships(driver)


C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\1603112739.py:29: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Linked L2 'Stores and Spares' -> L4 'Inventory Oils, Grease & Chemicals'
Linked L2 'Stores and Spares' -> L4 'Inventorie Consumable Stores & Misc.'
Linked L2 'Stores and Spares' -> L4 'G-Stock Materiali Ricoperti CBN'
Linked L2 'Stores and Spares' -> L4 'Stock in transit - Stores & Spares'
Linked L2 'Stores and Spares' -> L4 'Inventory Stores- Manual Adj'
Linked L2 'Stores and Spares' -> L4 'Inventory Packing Material'
Linked L2 'Inventory Machinery Spares' -> L4 'Inventory Spares'
Linked L2 'Inventory Machinery Spares' -> L4 'Inventory Manufactured Tools'
Linked L2 'Inventory Machinery Spares' -> L4 'Inventory Spares- Manual Adj'
Linked L2 'Inventory Machinery Spares' -> L4 'In Transit - Stores & Spares'
Linked L2 'Tools' -> L4 'Inventory Tools'
Linked L2 'Tools' -> L4 'Inventory Manufactured Tools'
Linked L2 'Tools' -> L4 'Inventory Tools- Manual Adj'
Linked L2 'Tools' -> L4 'Stock in transit - Tools'
Linked L2 'Tools' -> L4 'Inventory Dies'
Linked L2 'Other Inventory' -> L4 'Food & 

In [17]:
group_cols = [col for col in inventories_fy23.columns if col not in ['L1', 'L2', 'L3', 'L4']]

print(group_cols)

['001-AS', '133-FES', '008-SWARAJ', '002-HO', 'Powerol', 'CE', 'Agri', '004-MDS', 'Synergy', '007-SBU', 'TW', '134-TPDS', '135-INTNL OPER', '138-CONT SOUR', '140-NON MRV', '141-SOURCING', '139-CORP']


In [2]:
def link_groupcompany_to_all_levels_with_actuals(df, year, driver):
    group_cols = [col for col in df.columns if col not in ['L1', 'L2', 'L3', 'L4']]
    with driver.session() as session:
        for idx, row in df.iterrows():
            levels = {
                'InventoryLevel1': str(row['L1']).strip() if pd.notna(row['L1']) and row['L1'] else None,
                'InventoryLevel2': str(row['L2']).strip() if pd.notna(row['L2']) and row['L2'] else None,
                'InventoryLevel3': str(row['L3']).strip() if pd.notna(row['L3']) and row['L3'] else None,
                'InventoryLevel4': str(row['L4']).strip() if pd.notna(row['L4']) and row['L4'] else None,
            }

            for company in group_cols:
                val = row[company]
                amount_str = str(val).replace(",", "").strip()
                if amount_str in ["-", "--", "–", ""]:
                    amount = 0.0
                else:
                    try:
                        amount = float(amount_str)
                    except Exception:
                        amount = 0.0

                # Link all levels (L1-L4) unconditionally
                for level_label, level_name in levels.items():
                    if not level_name:
                        continue
                    session.run(f"""
                        MERGE (c:GroupCompany {{name: $company}})
                        MERGE (inv:{level_label} {{name: $inv_name}})
                        MERGE (c)-[r:HAS_AMOUNT {{year: $year}}]->(inv)
                        SET r.amount = $amount
                    """, company=company, inv_name=level_name, year=year, amount=amount)
                    print(f"[{level_label}] Linked {company} -> {level_name} with amount {amount} for {year}")





In [3]:
# Example usage:
# link_groupcompany_to_all_levels_with_actuals(inventories_fy23, "FY23", driver)
# link_groupcompany_to_all_levels_with_actuals(inventories_fy24, "FY24", driver)

In [19]:
import re

original_names = [
    "001-AS", "002-HO", "004-MDS", "007-SBU", "008-SWARAJ",
    "133-FES", "134-TPDS", "135-INTNL OPER", "138-CONT SOUR",
    "139-CORP", "140-NON MRV", "141-SOURCING"
]

def strip_number_prefix(name):
    return re.sub(r"^\d+\-?", "", name).strip()

cleaned_names = {old: strip_number_prefix(old) for old in original_names}

# Print mapping
print("Old to new mapping:")
for old, new in cleaned_names.items():
    print(f"{old} → {new}")

# Update in Neo4j
def update_groupcompany_names(driver):
    with driver.session() as session:
        for old_name, new_name in cleaned_names.items():
            session.run("""
                MATCH (c:GroupCompany {name: $old_name})
                SET c.name = $new_name
            """, old_name=old_name, new_name=new_name)
            print(f"Renamed: {old_name} → {new_name}")


update_groupcompany_names(driver)

Old to new mapping:
001-AS → AS
002-HO → HO
004-MDS → MDS
007-SBU → SBU
008-SWARAJ → SWARAJ
133-FES → FES
134-TPDS → TPDS
135-INTNL OPER → INTNL OPER
138-CONT SOUR → CONT SOUR
139-CORP → CORP
140-NON MRV → NON MRV
141-SOURCING → SOURCING
Renamed: 001-AS → AS
Renamed: 002-HO → HO
Renamed: 004-MDS → MDS
Renamed: 007-SBU → SBU
Renamed: 008-SWARAJ → SWARAJ
Renamed: 133-FES → FES
Renamed: 134-TPDS → TPDS
Renamed: 135-INTNL OPER → INTNL OPER
Renamed: 138-CONT SOUR → CONT SOUR
Renamed: 139-CORP → CORP
Renamed: 140-NON MRV → NON MRV
Renamed: 141-SOURCING → SOURCING


C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\1836264083.py:21: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [21]:
## Traded Goods............. ---->>>>  Traded Goods
def rename_inventorylevel3_node(driver, old_name, new_name):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (n:InventoryLevel3 {name: $old_name})
            SET n.name = $new_name
            RETURN count(n) AS updated_count
            """,
            old_name=old_name,
            new_name=new_name,
        )
        count = result.single()["updated_count"]
        if count:
            print(f"Renamed {count} InventoryLevel3 node(s) from '{old_name}' to '{new_name}'")
        else:
            print(f"No InventoryLevel3 node found with name '{old_name}'.")

# Usage:
rename_inventorylevel3_node(
    driver,
    "Stock In Transit - Traded Goods.............",
    "Stock In Transit - Traded Goods"
)


Renamed 1 InventoryLevel3 node(s) from 'Stock In Transit - Traded Goods.............' to 'Stock In Transit - Traded Goods'


C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\3725075887.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


## Quering through our Neo4j Graph

In [ ]:
## Querying through our Knowledge Graph 
def run_cypher_query(cypher_query, parameters=None):
    with driver.session() as session:
        result = session.run(cypher_query, parameters or {})
        # Convert Neo4j result to list of dicts, then to DataFrame
        records = [record.data() for record in result]
    return pd.DataFrame(records)


In [ ]:
def run_cypher_scalar(cypher, params=None):
    with driver.session() as session:
        result = session.run(cypher, params or {})
        raw = [record.data() for record in result]
        return raw

In [ ]:
## Get All Relationships

cypher = """
MATCH (g:GroupCompany)-[r:HAS_INVENTORY]->(i:InventoryType)
RETURN g.name AS company, i.name AS inventory_type, r.year AS year, r.amount AS amount
ORDER BY company, year, inventory_type
"""
df_all = run_cypher_query(cypher)
print(df_all.head())



In [ ]:
## Get Inventory for a Specific Company and Year
cypher = """
MATCH (g:GroupCompany {name: $company})-[r:HAS_INVENTORY {year: $year}]->(i:InventoryType)
RETURN g.name AS company, i.name AS inventory_type, r.year AS year, r.amount AS amount
ORDER BY inventory_type
"""
params = {"company": "001-AS", "year": "FY24"}
df_company = run_cypher_query(cypher, params)
print(df_company)

In [ ]:
## Get Total Inventory by Company for FY24
cypher = """
MATCH (g:GroupCompany)-[r:HAS_INVENTORY {year:"FY24"}]->(i:InventoryType)
RETURN g.name AS company, SUM(r.amount) AS total_inventory
ORDER BY total_inventory DESC
"""
df_total = run_cypher_query(cypher)
print(df_total)


In [ ]:
## Calculate the difference between a inventory for a company for FY23 - FY24

def year_difference(company, inventory):
    cypher = """
    MATCH (g:GroupCompany {name: $company})-[r:HAS_INVENTORY]->(i:InventoryType {name: $inventory})
    WHERE r.year IN ['FY24', 'FY23']
    RETURN
      MAX(CASE WHEN r.year = 'FY24' THEN r.amount ELSE 0 END) AS FY24,
      MAX(CASE WHEN r.year = 'FY23' THEN r.amount ELSE 0 END) AS FY23,
      MAX(CASE WHEN r.year = 'FY24' THEN r.amount ELSE 0 END) - 
      MAX(CASE WHEN r.year = 'FY23' THEN r.amount ELSE 0 END) AS difference
    """
    params = {
        "company": company,
        "inventory": inventory
    }
    df = run_cypher_query(cypher, params)
    return df

# Example usage:
df_diff = year_difference(
    company="001-AS",
    inventory="Raw Material"
)
print(df_diff)


In [ ]:
def delete_old_inventory_nodes(driver):
    with driver.session() as session:
        # Delete all InventoryType nodes and their relationships
        session.run("MATCH (n:InventoryType) DETACH DELETE n")
        print("Deleted all InventoryType nodes and their relationships.")
        
        # Delete all Inventories nodes and their relationships
        session.run("MATCH (n:Inventories) DETACH DELETE n")
        print("Deleted all Inventories nodes and their relationships.")

# Usage:
delete_old_inventory_nodes(driver)


## Get the schema of the KG

In [22]:
def save_kg_overview_to_markdown(driver, filename="kg_schema_overview3.md"):
    md = []
    with driver.session() as session:
        # Node labels
        md.append("## Node Labels\n")
        labels = [record[0] for record in session.run("CALL db.labels()")]
        for label in labels:
            md.append(f"- `{label}`")
        md.append("")

        # Relationship types
        md.append("## Relationship Types\n")
        rel_types = [record[0] for record in session.run("CALL db.relationshipTypes()")]
        for rel_type in rel_types:
            md.append(f"- `{rel_type}`")
        md.append("")

        # Relationship patterns
        md.append("## Relationship Patterns\n")
        patterns = session.run("""
            MATCH (a)-[r]->(b)
            RETURN DISTINCT labels(a) AS from_labels, type(r) AS rel_type, labels(b) AS to_labels
        """)
        for record in patterns:
            from_labels = ', '.join(record['from_labels'])
            to_labels = ', '.join(record['to_labels'])
            md.append(f"- `{from_labels}` -[`{record['rel_type']}`]-> `{to_labels}`")
        md.append("")

        # All nodes by label
        md.append("## All Nodes by Label\n")
        for label in labels:
            nodes = session.run(f"MATCH (n:`{label}`) RETURN n.name AS name ORDER BY name")
            names = [record["name"] for record in nodes if record["name"]]
            if names:
                md.append(f"### {label}")
                for name in names:
                    md.append(f"- {name}")
                md.append("")

        # All relationships (without values)
        md.append("## All Relationships (excluding actual values)\n")
        rels = session.run("""
            MATCH (a)-[r]->(b)
            RETURN type(r) AS rel_type, labels(a) AS from_labels, a.name AS from_name, labels(b) AS to_labels, b.name AS to_name
        """)
        for record in rels:
            md.append(f"- ({record['from_labels'][0]}: {record['from_name']}) -[{record['rel_type']}]-> ({record['to_labels'][0]}: {record['to_name']})")
        md.append("")

        # Group company names
        md.append("## All GroupCompany Nodes\n")
        group_companies = session.run("MATCH (n:GroupCompany) RETURN n.name AS name ORDER BY name")
        for record in group_companies:
            md.append(f"- {record['name']}")
        md.append("")

    # Save to file
    with open(filename, "w", encoding="utf-8") as f:
        f.write('\n'.join(md))
    print(f"KG schema overview saved to {filename}")

# Usage:
save_kg_overview_to_markdown(driver)


KG schema overview saved to kg_schema_overview3.md


C:\Users\emerg\AppData\Local\Temp\ipykernel_2656\2753191252.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
def save_has_amount_properties_to_markdown(driver, filename="has_amount_properties.md"):
    md = ["## Property Keys for Nodes in HAS_AMOUNT Relationships\n"]
    with driver.session() as session:
        result = session.run("""
            MATCH (a)-[r:HAS_AMOUNT]->(b)
            RETURN DISTINCT labels(a) AS labels_a, a AS node_a, labels(b) AS labels_b, b AS node_b
        """)

        properties_by_label = {}

        for record in result:
            for node, labels in [(record['node_a'], record['labels_a']), (record['node_b'], record['labels_b'])]:
                for label in labels:
                    if label not in properties_by_label:
                        properties_by_label[label] = set()
                    properties_by_label[label].update(node.keys())

        for label, props in properties_by_label.items():
            md.append(f"### {label}")
            for prop in sorted(props):
                md.append(f"- {prop}")
            md.append("")

    with open(filename, "w", encoding="utf-8") as f:
        f.write('\n'.join(md))
    print(f"Properties of nodes in HAS_AMOUNT relationships saved to {filename}")

# Usage:
save_has_amount_properties_to_markdown(driver)


In [ ]:
### Get list of all nodes and relationship for L2 and L3

def get_l2_l3_nodes_and_relationships(driver):
    with driver.session() as session:
        query = """
        MATCH (l2:InventoryLevel2)-[r:HAS_SUB_INVENTORY]->(l3:InventoryLevel3)
        RETURN l2.name AS Level2, type(r) AS RelType, l3.name AS Level3
        ORDER BY l2.name, l3.name
        """
        results = session.run(query)
        connections = []
        for record in results:
            l2 = record["Level2"]
            rel = record["RelType"]
            l3 = record["Level3"]
            connections.append((l2, rel, l3))
            print(f"{l2} -[{rel}]-> {l3}")
        return connections

# Usage:
get_l2_l3_nodes_and_relationships(driver)


In [ ]:
### Get list of all nodes and relationship for L3 and L4

def get_l3_l4_nodes_and_relationships(driver):
    with driver.session() as session:
        query = """
        MATCH (l3:InventoryLevel3)-[r:HAS_SUB_INVENTORY]->(l4:InventoryLevel4)
        RETURN l3.name AS Level3, type(r) AS RelType, l4.name AS Level4
        ORDER BY l3.name, l4.name
        """
        results = session.run(query)
        connections = []
        for record in results:
            l3 = record["Level3"]
            rel = record["RelType"]
            l4 = record["Level4"]
            connections.append((l3, rel, l4))
            print(f"{l3} -[{rel}]-> {l4}")
        return connections

# Usage:
get_l3_l4_nodes_and_relationships(driver)


In [ ]:
### Get list of all nodes and relationship for L4 and GroupCompanies

def get_l4_groupcompany_relationships(driver):
    with driver.session() as session:
        query = """
        MATCH (gc:GroupCompany)-[r:HAS_AMOUNT]->(l4:InventoryLevel4)
        RETURN gc.name AS GroupCompany, type(r) AS RelType, l4.name AS Level4, r.year AS Year, r.amount AS Amount
        ORDER BY l4.name, gc.name, r.year
        """
        results = session.run(query)
        connections = []
        for record in results:
            gc = record["GroupCompany"]
            rel = record["RelType"]
            l4 = record["Level4"]
            year = record["Year"]
            amount = record["Amount"]
            connections.append((gc, rel, l4, year, amount))
            print(f"{gc} -[{rel} {{year: {year}, amount: {amount}}}]-> {l4}")
        return connections

# Usage:
get_l4_groupcompany_relationships(driver)


## Closing the connection

In [ ]:
driver.close()

## More sample queries

In [ ]:
def run_query(query, parameters=None):
    with driver.session() as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]

In [ ]:
# 1. All InventoryLevel4 for GroupCompany 'AS'
query1 = """
MATCH (g:GroupCompany {name: "AS"})-[:HAS_AMOUNT]->(inv4:InventoryLevel4)
RETURN inv4.name
"""
results1 = run_query(query1)
print("InventoryLevel4 for AS:", results1)

In [ ]:
# 3. InventoryLevel4 linked to InventoryLevel2 'Stores and Spares'
query3 = """
MATCH (l2:InventoryLevel2 {name: "Stores and Spares"})-[:HAS_SUB_INVENTORY]->(l4:InventoryLevel4)
RETURN l4.name
"""
results3 = run_query(query3)
print("Level4 under 'Stores and Spares':", results3)

In [ ]:
# 4. GroupCompanies with HAS_AMOUNT to InventoryLevel4 'Inventory Spares'
query4 = """
MATCH (g:GroupCompany)-[:HAS_AMOUNT]->(inv4:InventoryLevel4 {name: "Inventory Spares"})
RETURN g.name
"""
results4 = run_query(query4)
print("Companies with 'Inventory Spares':", results4)

In [ ]:
examples = [
    {"node_name": "AS", "label": "GroupCompany"},
    {"node_name": "Inventories", "label": "InventoryLevel1"},
    {"node_name": "Stores and Spares", "label": "InventoryLevel2"},
    {"node_name": "Raw Material & Mfg. Componet", "label": "InventoryLevel3"},
    {"node_name": "Inventory Spares", "label": "InventoryLevel4"},
]

cypher_query = """
MATCH (n)-[r]-(m)
WHERE n.name = $node_name AND labels(n)[0] = $label
RETURN 
  n.name AS source_name, 
  labels(n)[0] AS source_label,
  type(r) AS relationship_type, 
  r.name AS rel_name, 
  r.year AS rel_year, 
  r.amount AS rel_amount, 
  m.name AS target_name, 
  labels(m)[0] AS target_label
"""

for example in examples:
    print(f"--- Connections for {example['label']} node '{example['node_name']}' ---")
    results = run_query(cypher_query, example)
    for record in results:
        print(record)
    print()


In [ ]:
cypher1 = """
MATCH (g:GroupCompany {name: 'Powerol'})-[:HAS_INVENTORY]->(i:InventoryType)
RETURN count(DISTINCT i) AS inventoryTypeCount
"""
answer1 = run_cypher_scalar(cypher1)
print(f"Number of inventory types for Powerol: {answer1}")

In [ ]:
cypher2 = """
MATCH (g:GroupCompany {name: '008-SWARAJ'})-[h:HAS_INVENTORY {year: 'FY24'}]->(i:InventoryType)
RETURN sum(h.amount) AS totalAmount
"""
answer2 = run_cypher_scalar(cypher2)
print(f"Total inventory amount for 008-SWARAJ in 2023: {answer2}")

In [ ]:
cypher3 = """
MATCH (g:GroupCompany {name: '001-AS'})-[h:HAS_INVENTORY]->(i:InventoryType)
WHERE i.name = 'Stores and Spares' AND h.year IN ['FY23', 'FY24']
RETURN 
  sum(CASE WHEN h.year = 'FY24' THEN h.amount ELSE 0 END) -
  sum(CASE WHEN h.year = 'FY23' THEN h.amount ELSE 0 END) AS difference
"""
answer3 = run_cypher_scalar(cypher3)
print(f"Difference in 'Stores and Spares' for 001-AS (FY24 - FY23): {answer3}")

In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY]->(i:InventoryType)
WHERE h.year IN ['FY24', 'FY23']
WITH i.name AS inventory_type,
     sum(CASE WHEN h.year = 'FY24' THEN h.amount ELSE 0 END) AS fy24_total,
     sum(CASE WHEN h.year = 'FY23' THEN h.amount ELSE 0 END) AS fy23_total
RETURN inventory_type, fy24_total, fy23_total, (fy24_total - fy23_total) AS difference
ORDER BY abs(difference) DESC
LIMIT 1
"""
with driver.session() as session:
    record = session.run(cypher).single()
    if record:
        print(f"Inventory type with highest change: {record['inventory_type']}")
        print(f"  FY24 Total: {record['fy24_total']}")
        print(f"  FY23 Total: {record['fy23_total']}")
        print(f"  Difference: {record['difference']}")
    else:
        print("No data found.")


In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY]->(i:InventoryType)
WHERE h.year IN ['FY24', 'FY23']
WITH i.name AS inventory_type,
     sum(CASE WHEN h.year = 'FY24' THEN h.amount ELSE 0 END) AS fy24_total,
     sum(CASE WHEN h.year = 'FY23' THEN h.amount ELSE 0 END) AS fy23_total
RETURN
  inventory_type,
  fy24_total,
  fy23_total,
  (fy24_total - fy23_total) AS difference
ORDER BY abs(difference) DESC
LIMIT 1
"""
with driver.session() as session:
    record = session.run(cypher).single()
    if record:
        print(f"Inventory type with the highest difference between FY24 and FY23: ")
        print({record['inventory_type']})
        print(f"  FY24 Total: {record['fy24_total']}")
        print(f"  FY23 Total: {record['fy23_total']}")
        print(f"  Difference (FY24 - FY23): {record['difference']}")
    else:
        print("No data found.")


In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY {year: 'FY23'}]->(i:InventoryType)
RETURN g.name AS company_name, i.name AS inventory_type, h.amount AS amount
ORDER BY company_name, inventory_type
"""

with driver.session() as session:
    results = session.run(cypher)
    for record in results:
        print(
            f"Company: {record['company_name']}, "
            f"Inventory: {record['inventory_type']}, "
            f"Amount: {record['amount']}"
        )


In [ ]:
def total_inventory_for_company_and_year(company_name, year):
    cypher = """
    MATCH (g:GroupCompany {name: $company_name})-[h:HAS_INVENTORY {year: $year}]->(i:InventoryType)
    RETURN sum(h.amount) AS totalAmount
    """
    params = {
        "company_name": company_name,
        "year": year
    }
    with driver.session() as session:
        result = session.run(cypher, params)
        record = result.single()
        return record["totalAmount"] if record else None

# Example usage:s
total = total_inventory_for_company_and_year("002-HO", "FY23")
print(f"Total inventory for 002-HO in FY24: {total}")


In [ ]:
cypher = """
MATCH (g:GroupCompany)-[h:HAS_INVENTORY {year: 'FY24'}]->(i:InventoryType {name: 'Raw Material'})
RETURN g.name AS company_name, h.amount AS raw_material_amount
ORDER BY raw_material_amount DESC
LIMIT 1
"""

with driver.session() as session:
    result = session.run(cypher)
    record = result.single()
    if record:
        print(f"Group company with highest Raw Material in FY24: {record['company_name']} ({record['raw_material_amount']})")
    else:
        print("No data found.")
